In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from time import sleep
import csv
import re

from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from tenacity import retry, wait_exponential, stop_after_attempt


In [ ]:
# Tạo đối tượng cấu hình cho trình duyệt Chrome, thiết lập tùy chọn khi khởi chạy Chrome bằng Selenium
chrome_options = webdriver.ChromeOptions()

# Khởi động gọi trình duyệt tự động bằng webdriver
service = Service('C:/Users/ADMIN/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
sleep(1)

driver.get('https://tiki.vn/')

sleep(30)

### Category

In [ ]:
# Lấy danh sách url_1
category_1 = driver.find_elements(By.CSS_SELECTOR, ".styles__StyledListItem-sc-w7gnxl-0.cjqkgR .styles__StyledItemV2-sc-oho8ay-1.bHIPhv a")
category_1 = category_1[:-3]

In [ ]:
category_urls = []
for element in category_1:
    url = element.get_attribute("href")
    category_urls.append(url) 

In [ ]:
found_a_tags_urls = []

for url in category_urls:
    driver.get(url)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "styles__TreeItemContainer-sc-1uq9a9i-1.kLOesk")))

    try:
        tree_item_containers = driver.find_elements(By.CLASS_NAME, "styles__TreeItemContainer-sc-1uq9a9i-1.kLOesk")
        
        for container in tree_item_containers:
            try:
                children = container.find_elements(By.XPATH, "./*")
                
                if len(children) > 0 and children[0].tag_name == "a":
                    url = children[0].get_attribute("href")
                    if url:
                        found_a_tags_urls.append(url)

                else:
                    try:
                        tree_icon = container.find_element(By.CLASS_NAME, "styles__TreeIcon-sc-1uq9a9i-4.kJopVc")
                        tree_icon.click()
                        sleep(3)

                        sub_items = container.find_elements(By.CLASS_NAME, "styles__TreeSubItem-sc-1uq9a9i-6.divFYE")
                        
                        for sub_item in sub_items:
                            try:
                                a_tag = sub_item.find_element(By.TAG_NAME, "a")
                                url = a_tag.get_attribute("href")
                                if url:
                                    found_a_tags_urls.append(url)
                            except Exception as e:
                                print(f"Không tìm thấy thẻ <a> trong sub-item: {e}")

                    except Exception as e:
                        print(f"Không tìm thấy hoặc không click được vào phần tử icon: {e}")

            except StaleElementReferenceException as e:
                print(f"Đã xảy ra lỗi StaleElementReferenceException: {e}")

    except Exception as e:
        print(f"Đã xảy ra lỗi với URL {url}: {e}")


In [ ]:
urls_with_class = []
urls_without_class = []

for url in found_a_tags_urls:
    driver.get(url)

    try:
        content_wrap = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "styles__ContentWrap-sc-ov39d-0.dnibXS"))
        )
        
        left_element = content_wrap.find_element(By.CSS_SELECTOR, ".styles__Left-sc-ov39d-2.gurqJq")
        
        urls_with_class.append(url)
    
    except:
        urls_without_class.append(url)

In [ ]:
urls_with_copy = urls_with_class.copy()
urls_without_copy = urls_without_class.copy()

In [ ]:
for url in urls_with_copy:
    driver.get(url)

    # Đợi phần tử chính được tải
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "styles__TreeItemContainer-sc-1uq9a9i-1.kLOesk"))
    )

    try:
        # Tìm tất cả phần tử có class "styles__TreeItemContainer-sc-1uq9a9i-1 kLOesk"
        tree_item_containers = driver.find_elements(By.CLASS_NAME, "styles__TreeItemContainer-sc-1uq9a9i-1.kLOesk")

        # Duyệt qua từng phần tử tree_item_container
        for container in tree_item_containers:
            try:
                # Tìm tất cả các phần tử con của container
                children = container.find_elements(By.XPATH, "./*")

                # Nếu phần tử chỉ có một con và nó là thẻ <a>, lấy URL từ thẻ a
                if len(children) == 1 and children[0].tag_name == "a":
                    url = children[0].get_attribute("href")
                    if url:
                        urls_without_copy.append(url)

                # Nếu phần tử có hai con, tìm và click vào icon để mở sub-items
                elif len(children) == 2:
                    try:
                        # Tìm icon và click
                        tree_icon = container.find_element(By.CLASS_NAME, "styles__TreeIcon-sc-1uq9a9i-4.kJopVc")
                        tree_icon.click()
                        sleep(3)  # Chờ sub-items tải

                        # Tìm tất cả các sub-item với class "styles__TreeSubItem-sc-1uq9a9i-6 divFYE"
                        sub_items = container.find_elements(By.CLASS_NAME, "styles__TreeSubItem-sc-1uq9a9i-6.divFYE")

                        # Duyệt qua từng sub-item và lấy URL từ thẻ <a>
                        for sub_item in sub_items:
                            try:
                                a_tag = sub_item.find_element(By.TAG_NAME, "a")
                                url = a_tag.get_attribute("href")
                                if url:
                                    urls_without_copy.append(url)
                            except Exception as e:
                                print(f"Không tìm thấy thẻ <a> trong sub-item: {e}")

                    except Exception as e:
                        print(f"Không tìm thấy hoặc không click được vào phần tử icon: {e}")

            except Exception as e:
                print(f"Lỗi khi xử lý phần tử container: {e}")

    except Exception as e:
        print(f"Đã xảy ra lỗi với URL {url}: {e}")

    if url in urls_with_class:
        urls_with_class.remove(url)
    

In [ ]:
urls_page_product = urls_without_copy.copy()

In [ ]:
product_url = []

for url in urls_page_product:
    driver.get(url)
    

    try:
        right_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "styles__Right-sc-ov39d-3.cxFVLR"))
        )
        loading_container = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "styles__StyledLoadingContainer-sc-1p0mhu9-0.dtOgzC"))
        )

        if loading_container:
            product_links = WebDriverWait(right_element, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.product-item'))
            )
            for link in product_links:
                href = link.get_attribute("href")
                if href:
                    product_url.append(href)
                    
    except Exception as e:
        continue

In [ ]:
product_url_copy = product_url.copy()

In [ ]:
import os
# Đường dẫn tới tệp CSV
csv_file_path = "url_category.csv"

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(["URLs"])
    
    for url in urls_without_copy:
        writer.writerow([url])

print(f"Tệp {csv_file_path} đã được tạo và lưu trữ thành công.")

In [ ]:
csv_file_path = "url_product.csv"

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(["URLs"])
    
    for url in product_url_copy:
        writer.writerow([url])

print(f"Tệp {csv_file_path} đã được tạo và lưu trữ thành công.")

In [ ]:
product_url_copy = []
file_path = 'url_products.csv'

with open(file_path, newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
        product_url_copy.append(row[0])


### Product

In [ ]:
num_subarrays = 30

total_elements = len(product_url_copy)
elements_per_subarray = total_elements // num_subarrays
remainder = total_elements % num_subarrays

subarrays = {}

index = 0
for i in range(num_subarrays):
    if i < remainder:
        subarray_size = elements_per_subarray + 1
    else:
        subarray_size = elements_per_subarray
    
    subarray_name = f'product_url_{i+1}'
    subarrays[subarray_name] = product_url_copy[index:index + subarray_size]
    index += subarray_size


In [ ]:
products = []

for idx, url in enumerate(example_subarray, start=2028):
    driver.get(url)

    product_data = []

    product_data.append(idx)
    product_data.append(url)

    try:
        breadcrumb_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "breadcrumb"))
        )
        breadcrumb_html = breadcrumb_element.get_attribute('innerHTML')
        soup = BeautifulSoup(breadcrumb_html, 'html.parser')
        breadcrumb_links = soup.find_all('a')[1:]

        type_text = " - ".join([a.find('span').text for a in breadcrumb_links])
        product_data.append(type_text)
    except TimeoutException:
        product_data.append("")

    try:
        body_info = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "styles__BodyInfoStyled-sc-smyo6b-3"))
        )
        soup = BeautifulSoup(body_info.get_attribute('innerHTML'), 'html.parser')

        try:
            name_product = soup.find('h1', class_="Title__TitledStyled-sc-c64ni5-0").text
            product_data.append(name_product)
        except AttributeError:
            product_data.append("")

        try:
            rating_product = soup.find('div', class_="styles__StyledReview-sc-1onuk2l-1").text
            product_data.append(rating_product)
        except AttributeError:
            product_data.append("")

        try:
            sales = soup.find('div', class_="styles__StyledQuantitySold-sc-1onuk2l-3").text
            product_data.append(sales)
        except AttributeError:
            product_data.append("")

        try:
            price_element = soup.find('div', class_="product-price__current-price")
            price = price_element.find(text=True, recursive=False).strip()
            product_data.append(price)
        except AttributeError:
            product_data.append("")

    except TimeoutException:
        product_data.extend(["", "", "", ""])

    try:
        seller_info = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "SellerHeader__SellerHeaderStyled-sc-la7c6v-0"))
        )
    
        seller_url = WebDriverWait(seller_info, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "a"))
        ).get_attribute('href')
    
        product_data.append(seller_url)

    except TimeoutException:
        product_data.append("")
    except NoSuchElementException:
        product_data.append("")
    
    products.append(product_data)

    sleep(2)

In [ ]:
header = ['id', 'url_product', 'type', 'name_product', 'rating_product', 'sales', 'price', 'seller_url']
products_copy.insert(0, header)

with open('products.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    writer.writerows(products_copy)

print("Đã lưu mảng products vào file products.csv với tiêu đề.")


In [ ]:
with open('products.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    writer.writerows(products_copy)


In [ ]:
# Hàm chờ cho phần tử xuất hiện trên trang đích
def wait_for_page_load(driver, timeout=10):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

for i, url in enumerate(product_url_copy):
    if url.startswith("https://tiki.vn/"):
        continue
    elif url.startswith("https://tka.tiki.vn/"):
        try:
            # Bắt lỗi nếu có sự cố trong driver.get(url)
            driver.get(url)
            wait_for_page_load(driver)
            final_url = driver.current_url
            
            # Thay thế URL trong mảng
            product_url_copy[i] = final_url
        except Exception as e:
            continue


In [ ]:
csv_file_path = "url_products.csv"

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(["URLs"])
    
    for url in product_url_copy:
        writer.writerow([url])

print(f"Tệp {csv_file_path} đã được tạo và lưu trữ thành công.")

In [ ]:
import re

api_products = []

# Hàm xử lý để lấy new_url từ từng loại URL
def process_url(url):
    # Loại 1: URL đơn giản với 'p<number>.html?spid=<number>'
    match = re.search(r'p(\d+)\.html\?spid=(\d+)', url)
    if match:
        product_id = match.group(1)
        pid = match.group(2)
        new_url = f"https://tiki.vn/api/v2/products/{product_id}?platform=web&spid={pid}&version=3"
        return new_url

    # Loại 2: URL phức tạp hơn với 'p<number>.html' và spid nằm sau nhiều query parameters
    match = re.search(r'p(\d+)\.html.*spid=(\d+)', url)
    if match:
        product_id = match.group(1)
        pid = match.group(2)
        new_url = f"https://tiki.vn/api/v2/products/{product_id}?platform=web&spid={pid}&version=3"
        return new_url

    return None

for url in product_url_copy:
    new_url = process_url(url)
    if new_url:
        api_products.append(new_url)
    else:
        continue


In [ ]:
import requests
import csv
import time
from tenacity import retry, wait_exponential, stop_after_attempt

product_list = []

# Hàm gửi yêu cầu HTTP với cơ chế retry
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5))
def get_product_data(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Lỗi HTTP {response.status_code}: {response.text}")

# Lặp qua từng URL trong api_products
for index, url in enumerate(api_products):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }

    try:
        # Gọi hàm với cơ chế retry
        data = get_product_data(url, headers)
        
        # Truy cập vào các thuộc tính mong muốn với việc xử lý trường hợp rỗng
        product_info = {
            "id": data.get("id") if data.get("id") is not None else "null",
            "name": data.get("name") if data.get("name") is not None else "null",
            "url": data.get("url") if data.get("url") is not None else "null",
            "short_description": data.get("short_description") if data.get("short_description") is not None else "null",
            "price": data.get("price") if data.get("price") is not None else "null",
            "rating_average": data.get("rating_average") if data.get("rating_average") is not None else "null",
            "review_count": data.get("review_count") if data.get("review_count") is not None else "null",
            "all_time_quantity_sold": data.get("all_time_quantity_sold") if data.get("all_time_quantity_sold") is not None else "null",
            "brand_id": data.get("brand", {}).get("id") if data.get("brand") else "null",
            "brand_name": data.get("brand", {}).get("name") if data.get("brand") else "null",
            "gift_item_title": data.get("gift_item_title") if data.get("gift_item_title") is not None else "null",
            "current_seller_id": (data.get("current_seller").get("id") if data.get("current_seller") else "null"),
            "current_seller_name": (data.get("current_seller").get("name") if data.get("current_seller") else "null"),
            "current_seller_link": (data.get("current_seller").get("link") if data.get("current_seller") else "null"),
            "current_seller_store_id": (data.get("current_seller").get("store_id") if data.get("current_seller") else "null"),
        }

        # Lấy tên từ breadcrumbs, bỏ tên cuối cùng và nối chúng lại với nhau
        breadcrumbs = data.get("breadcrumbs", [])
        breadcrumb_names = [breadcrumb.get("name") for breadcrumb in breadcrumbs]
        
        # Bỏ tên cuối cùng
        if breadcrumb_names:
            breadcrumb_names = breadcrumb_names[:-1]  # Xóa tên cuối cùng
        
        product_info["breadcrumbs"] = ", ".join(breadcrumb_names) if breadcrumb_names else "null"

        # Thêm thông tin sản phẩm vào danh sách
        product_list.append(product_info)

    except Exception as e:
        print(f"Lỗi khi truy cập URL {url}: {str(e)}")
        continue


In [ ]:
import re

product_id = []

for url in product_url_copy:
    match = re.search(r'p(\d+)', url)
    if match:
        product_id_value = match.group(1)
        product_id.append((product_id_value, url))


In [ ]:
product_id_dict = {pid: url for pid, url in product_id}

for product in product_list_copy:
    product_id_value = product.get('id') 
    if product_id_value in product_id_dict:
        product['url'] = product_id_dict[product_id_value]
        

In [ ]:
# Ghi danh sách sản phẩm vào file CSV
with open('product_list.csv', mode='w', newline='', encoding='utf-8') as csv_file:
    # Lấy các khóa từ sản phẩm đầu tiên để làm tiêu đề cột
    fieldnames = product_list_copy[0].keys() if product_list_copy else []
    
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()  # Ghi tiêu đề cột
    for product in product_list_copy:
        writer.writerow(product)  # Ghi từng sản phẩm vào file

print("Dữ liệu đã được lưu vào file product_list.csv.")

### Store

In [ ]:
stores = []

seen_sellers = set()

for product in product_list_copy:
    seller_id = product.get('current_seller_id')
    
    if seller_id not in seen_sellers:
        store_info = {
            'seller_id': seller_id,
            'seller_name': product.get('current_seller_name'),
            'seller_link': product.get('current_seller_link'),
            'seller_store_id': product.get('current_seller_store_id'),
        }
        stores.append(store_info)
        seen_sellers.add(seller_id)
        

In [ ]:
cleaned_stores = [store for store in stores if store['seller_link'] != 'null']

In [ ]:
for store in cleaned_stores:
    store['api_url'] = f"https://api.tiki.vn/social/openapi/interaction/following?tiki_seller_id={store['seller_id']}"
    

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5))
def get_follower_data(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Lỗi HTTP {response.status_code}: {response.text}")

for store in cleaned_stores:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }

    try:
        data = get_follower_data(store['api_url'], headers)
        
        total_follower = data.get("data", {}).get("following", {}).get("total_follower", 0) 
        
        store['total_follower'] = total_follower

    except RequestException as e:
        print(f"Không thể truy cập API cho seller_id {store['seller_id']}: {e}")
        store['total_follower'] = None
        continue



In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5))
def get_seller_rating(url):
    driver.get(url)
    try:
        rating_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "Header__SubInfo-sc-vgcg69-12.iUmIsA"))
        )
        return rating_element.text.strip().split('/')[0]
    except TimeoutException:
        print(f"Timeout: Không tìm thấy phần tử rating cho {url}")
        return None 
    except Exception as e:
        print(f"Lỗi: {e}")
        return None 

for store in cleaned_stores_copy:
    try:
        rating = get_seller_rating(store['seller_link'])
        store['rating_seller'] = rating 
    except Exception as e:
        print(f"Không thể lấy rating cho {store['seller_link']}: {e}")
        store['rating_seller'] = None 


In [ ]:
# Ghi danh sách cửa hàng vào file CSV
with open('store_list.csv', mode='w', newline='', encoding='utf-8') as csv_file:
    # Lấy các khóa từ sản phẩm đầu tiên để làm tiêu đề cột
    fieldnames = cleaned_stores_copy[0].keys() if cleaned_stores_copy else []
    
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()  # Ghi tiêu đề cột
    for store in cleaned_stores_copy:
        writer.writerow(store)  # Ghi từng sản phẩm vào file

print("Dữ liệu đã được lưu vào file store_list.csv.")

### Review

In [ ]:
api_template = "https://tiki.vn/api/v2/reviews?limit=20&include=comments,contribute_info,attribute_vote_summary&sort=score%7Cdesc,id%7Cdesc,stars%7Call&page=1&spid={spid}&product_id={product_id}&seller_id={seller_id}"

api_reviews = []

for product in product_list_copy:
    product_id = product.get('id')
    
    spid_match = re.search(r'spid=(\d+)', product.get('url', ''))
    spid = spid_match.group(1) if spid_match else 'Unknown'

    # Lấy seller_id từ current_seller_id
    seller_id = product.get('current_seller_id', 'Unknown')

    # Tạo API cho từng sản phẩm
    api_url = api_template.format(spid=spid, product_id=product_id, seller_id=seller_id)
    
    # Lưu API vào danh sách
    api_reviews.append(api_url)


In [ ]:
total_elements = len(api_reviews) 
n = 10 
chunk_size = total_elements // n
remainder = total_elements % n

api_reviews_chunks = []

for i in range(n):
    start_index = i * chunk_size
    end_index = start_index + chunk_size
    api_reviews_chunks.append(api_reviews[start_index:end_index])

# Phân phối phần dư vào phần cuối cùng
if remainder > 0:
    api_reviews_chunks[-1] += api_reviews[-remainder:]


In [ ]:
import requests
from tenacity import retry, wait_exponential, stop_after_attempt

@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5))
def get_product_data(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Lỗi HTTP {response.status_code}: {response.text}")

def process_review_data(data):
    if data is None:
        return None
    
    created_by = data.get("created_by", {})
    contribute_info = created_by.get("contribute_info", {})
    summary = contribute_info.get("summary", {})
    
    return {
        "id": data.get("id") if data.get("id") is not None else "null",
        "title": data.get("title") if data.get("title") is not None else "null",
        "content": data.get("content") if data.get("content") is not None else "null",
        "status": data.get("status") if data.get("status") is not None else "null",
        "rating": data.get("rating") if data.get("rating") is not None else "null",
        "created_at": data.get("created_at") if data.get("created_at") is not None else "null",
        
        "created_by_id": created_by.get("id") if created_by.get("id") is not None else "null",
        "created_by_name": created_by.get("name") if created_by.get("name") is not None else "null",
        "created_by_full_name": created_by.get("full_name") if created_by.get("full_name") is not None else "null",
        "created_by_created_time": created_by.get("created_time") if created_by.get("created_time") is not None else "null",
        "created_by_group_id": created_by.get("group_id") if created_by.get("group_id") is not None else "null",
        "created_by_purchased": created_by.get("purchased") if created_by.get("purchased") is not None else "null",
        "created_by_purchased_at": created_by.get("purchased_at") if created_by.get("purchased_at") is not None else "null",
        
        "contribute_info_joined_time": summary.get("joined_time") if summary.get("joined_time") is not None else "null",
        "contribute_info_total_review": summary.get("total_review") if summary.get("total_review") is not None else "null",
        "contribute_info_total_thank": summary.get("total_thank") if summary.get("total_thank") is not None else "null",
        
        "product_id": data.get("product_id") if data.get("product_id") is not None else "null",
        "timeline_review_created_date": data.get("timeline", {}).get("review_created_date") if data.get("timeline", {}).get("review_created_date") is not None else "null",
        "timeline_delivery_date": data.get("timeline", {}).get("delivery_date") if data.get("timeline", {}).get("delivery_date") is not None else "null",
    }

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}

reviews_list = [] 

for base_url in api_reviews_chunks[9]:
    page = 1
    while True:
        url = re.sub(r'page=\d+', f'page={page}', base_url)
        
        try:
            data = get_product_data(url, headers)
            if data.get("data", []):
                reviews = data.get("data")
            else:
                break
            
            for review in reviews:
                processed_review = process_review_data(review)
                reviews_list.append(processed_review)
            
            page += 1

        except Exception as e:
            break

In [ ]:
with open('review_list.csv', mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = reviews_list_copy[0].keys() if reviews_list_copy else []
    
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for review in reviews_list_copy:
        writer.writerow(review)

print("Dữ liệu đã được lưu vào file review_list.csv.")

### Customer

In [ ]:
import pandas as pd
from datetime import datetime

customer_data = []

for review in review_list_copy:
    customer_info = {
        'created_by_id': review['created_by_id'],
        'created_by_name': review['created_by_name'],
        'created_by_full_name': review['created_by_full_name'],
        'created_by_created_time': review['created_by_created_time'],
        'created_by_group_id': review['created_by_group_id'],
        'contribute_info_joined_time': review['contribute_info_joined_time'],
        'contribute_info_total_review': review['contribute_info_total_review'],
        'contribute_info_total_thank': review['contribute_info_total_thank']
    }
    customer_data.append(customer_info)

df = pd.DataFrame(customer_data)

df['created_by_created_time'] = pd.to_datetime(df['created_by_created_time'], errors='coerce')

customer_list = df.sort_values(by=['created_by_created_time'], ascending=False).drop_duplicates(subset='created_by_id', keep='first')


In [ ]:
customer_list.to_csv('customer_list.csv', index=False)

print("File customer_list.csv đã được lưu thành công.")


### Orders

In [ ]:
order_data = []

for review in review_list_copy:
    order_info = {
        'id': review['id'], 
        'created_by_id': review['created_by_id'], 
        'created_by_name': review['created_by_name'],
        'product_id': review['product_id'], 
        'timeline_delivery_date': review['timeline_delivery_date']
    }
    order_data.append(order_info)

order_list = pd.DataFrame(order_data)

order_list['timeline_delivery_date'] = pd.to_datetime(order_list['timeline_delivery_date'], errors='coerce')


In [ ]:
order_list.to_csv('order_list.csv', index=False)

print("File order_list.csv đã được lưu thành công.")
